In [ ]:
import boto3
from botocore.exceptions import NoCredentialsError

# AWS 접속 설정
BUCKET_NAME = 'clean-sfx-audios'

region_name = 'us-west-2'  # Replace with your desired AWS region
# Create a session
session = boto3.Session(
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    region_name=region_name
)
# Create an S3 client
s3 = session.resource("s3")
s3_client=session.client("s3")
s3_client.download_file(BUCKET_NAME, "pond5/total_total_csv.csv", "total_total_csv.csv")

In [ ]:
#setup aws s3
#authorize for boto3
#optimizer ai account
import boto3
region_name = 'us-west-2'  # Replace with your desired AWS region
# AWS 접속 설정
BUCKET_NAME = 'clean-sfx-audios'

# Create a session
session = boto3.Session(
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    region_name=region_name
)
# Create an S3 client
s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY, aws_secret_access_key=SECRET_KEY)

def upload_file_to_s3(upload):
    try:
        response = s3.upload_file(upload['upload_file'], 'clean-sfx-audios', upload['upload_path'])
    except NoCredentialsError:
        print("Credentials not available")
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
response = s3.upload_file("total_0403.csv", 'clean-sfx-audios', "pond/updated_filtered_csv_0403.csv")

In [ ]:
import pandas as pd
from multiprocessing import Pool
from tqdm import tqdm

additional_total = pd.read_csv("additional_total.csv")

uploads = []
for i in tqdm(range(len(additional_total))):
    data = additional_total.iloc[i]
    if data['folder'] in ['data18','data19','data20']:
        upload_file = data['folder'] + "/" + data['download_id']+".wav"
        upload_path = "pond/pond_7/"+data['download_id']+".wav"
        uploads.append({
            "upload_file": upload_file,
            "upload_path": upload_path
        })
        
        if len(uploads) == 384:
            with Pool(384) as p:
                p.map(upload_file_to_s3, uploads)
                uploads = []

In [1]:
from datasets import load_dataset

ds = load_dataset("ylacombe/expresso")

from tqdm import tqdm
styles = set()
for data in tqdm(ds['train']):
    styles.add(data['style'])

README.md:   0%|          | 0.00/4.07k [00:00<?, ?B/s]

train-00000-of-00012.parquet:   0%|          | 0.00/473M [00:00<?, ?B/s]

train-00001-of-00012.parquet:   0%|          | 0.00/582M [00:00<?, ?B/s]

train-00002-of-00012.parquet:   0%|          | 0.00/505M [00:00<?, ?B/s]

train-00003-of-00012.parquet:   0%|          | 0.00/466M [00:00<?, ?B/s]

...

train-00009-of-00012.parquet:   0%|          | 0.00/428M [00:00<?, ?B/s]

train-00010-of-00012.parquet:   0%|          | 0.00/513M [00:00<?, ?B/s]

train-00011-of-00012.parquet:   0%|          | 0.00/450M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11615 [00:00<?, ? examples/s]

100%|██████████| 11615/11615 [00:29<00:00, 391.30it/s]


In [3]:
from tqdm import tqdm
import soundfile as sf
import numpy as np
import os

datas = [['audio_path', 'caption', 'duration', 'script']]
# 데이터셋 순회하면서 오디오 파일 저장
num = 0
for data in tqdm(ds['train']):
    audio_data = data['audio']['array']
    duration = data['audio']['array'].shape[0]/data['audio']['sampling_rate']
    save_directory = '/workspace/data/expresso'

    if data['speaker_id'] == 'ex01':
        gender = 'person'
    elif data['speaker_id'] == 'ex02':
        gender = 'woman'
    elif data['speaker_id'] == 'ex03':
        gender = 'man'
    elif data['speaker_id'] == 'ex04':
        gender = 'woman'

    text = data['text']
    if data['style'] == 'confused':
        caption = f'a {gender} saying in a confused tone'
    elif data['style'] == 'happy':
        caption = f'a {gender} happily saying'
    elif data['style'] == 'laughing':
        caption = f'a {gender} laughing and saying'
    elif data['style'] == 'sad':
        caption = f'a {gender} saying in a sad tone'
    elif data['style'] == 'whisper':
        num += 1
        caption = f'a {gender} whispering'
    elif data['style'] == 'enunciated':
        caption = f'a {gender} clearly saying'
    elif data['style'] == 'singing':
        caption = f'a {gender} singing'
    elif data['style'] == 'essentials':
        caption = f'a {gender} essentialy saying'
    elif data['style'] == 'emphasis':
        caption = f'a {gender} clearly saying'
    else:
        caption = f'a {gender} saying'

    sampling_rate = data['audio']['sampling_rate']
    
    # Mono인 경우 Stereo로 변환
    if audio_data.ndim == 1:
        audio_data = np.stack([audio_data, audio_data], axis=1)
    
    # 저장 경로 설정
    save_path = os.path.join(save_directory, data['audio']['path'])
    
    # 오디오 파일 저장
    sf.write(save_path, audio_data, sampling_rate)

    # datas.append([save_path, caption, duration, text])
print(num)

100%|██████████| 11615/11615 [01:21<00:00, 142.50it/s]

1518


In [ ]:
# huggingface-cli login
from huggingface_hub import HfApi

api = HfApi()

In [ ]:
api.upload_file(
    path_or_fileobj="/workspace/dit_audio/voicefx/weights_vsfx_1108/model_69.pth",
    path_in_repo="speech_1108_model_69.pth",
    repo_id="Daniel777/base_38",
    repo_type="model"
)

In [1]:
import pandas as pd

df = pd.read_csv('./datas/voice_sfx.csv')

In [14]:
df = df.dropna()
df['caption'] = df['new_caption']
df.to_csv('./voice_sfx.csv', index=False)